In [1]:
#english to French SEQ2SEQ model

In [2]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from keras.callbacks import ModelCheckpoint
import nltk
import numpy as np
import os
import zipfile
import sys
import urllib.request
from gensim.models import KeyedVectors
from keras.utils import plot_model


Using TensorFlow backend.
C:\Users\santh_000\Anaconda3\envs\tfp3.6\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [53]:
from keras.activations import softmax
from keras.layers.core import Dense, Activation, RepeatVector, Permute
from keras.layers import Input, Embedding, Multiply, Concatenate, Lambda
from keras.layers.wrappers import TimeDistributed

#hyperparameters

In [18]:
BATCH_SIZE = 64
NUM_EPOCHS = 100
HIDDEN_UNITS = 256
NUM_SAMPLES = 10000
MAX_VOCAB_SIZE = 10000
EMBEDDING_SIZE = 100
DATA = 'fra.txt'#loading data from default work directory

In [19]:
tar_count = Counter()

GLOVE_MODEL = "glove.6B.100d.txt"
WEIGHT_FILE_PATH = 'eng-to-fr-glove-weights.h5'


In [20]:
def load_glove():
    
    _word2em = {}
    file = open(GLOVE_MODEL, mode='r', encoding='utf8')
    for line in file:
        words = line.strip().split()
        word = words[0]
        embeds = np.array(words[1:], dtype=np.float32)
        _word2em[word] = embeds
    file.close()
    return _word2em

In [21]:
word2em = load_glove()

In [22]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: len(lines)-1]:
    input_text, target_text = line.split('\t')
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    target_text = '\t' + target_text + '\n'
    for char in target_text:
        tar_count[char] += 1


In [23]:
target_word2idx = dict()

for idx, word in enumerate(tar_count.most_common(MAX_VOCAB_SIZE)):
    #print(word)
    target_word2idx[word[0]] = idx

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_decoder_tokens = len(target_idx2word)

In [24]:
unknown_emb = np.random.randn(EMBEDDING_SIZE)

In [25]:
encoder_max_seq_length = 0
decoder_max_seq_length = 0

input_texts_word2em = []


In [26]:
#encoder word2index input

In [27]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    encoder_input_wids = []
    for w in input_words:
        em = unknown_emb
        if w in word2em:
            em = word2em[w]
        encoder_input_wids.append(em)
    input_texts_word2em.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_text), decoder_max_seq_length)

In [28]:
encoder_input_data = pad_sequences(input_texts_word2em, encoder_max_seq_length)

In [29]:
#decoder word2index input

In [37]:
decoder_target_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
decoder_input_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
lines = open(DATA, 'rt', encoding='utf8').read().split('\n')
for lineIdx, line in enumerate(lines[: min(NUM_SAMPLES, len(lines)-1)]):
    _, target = line.split('\t')
    target = '\t' + target + '\n'
    for idx, char in enumerate(target):
        if char in target_word2idx:
            w2idx = target_word2idx[char]
            decoder_input_data[lineIdx, idx, w2idx] = 1
            if idx > 0:
                decoder_target_data[lineIdx, idx-1, w2idx] = 1


In [49]:
context = dict()
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length


In [50]:
#defining Encoder- Decoder Model

In [54]:
encoder_inputs = Input(shape=(None, EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [40]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')



checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
          verbose=1, validation_split=0.3, callbacks=[checkpoint])

model.save_weights(WEIGHT_FILE_PATH)

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
7000/7000 [==============================] - 86s 12ms/step - loss: 0.9723 - val_loss: 1.0034


C:\Users\santh_000\Anaconda3\envs\tfp3.6\lib\site-packages\keras\engine\topology.py:2368: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/100
7000/7000 [==============================] - 78s 11ms/step - loss: 0.7787 - val_loss: 0.8206
Epoch 3/100
7000/7000 [==============================] - 77s 11ms/step - loss: 0.6341 - val_loss: 0.7187
Epoch 4/100
7000/7000 [==============================] - 77s 11ms/step - loss: 0.5636 - val_loss: 0.6648
Epoch 5/100
7000/7000 [==============================] - 76s 11ms/step - loss: 0.5230 - val_loss: 0.6357
Epoch 6/100
7000/7000 [==============================] - 76s 11ms/step - loss: 0.4947 - val_loss: 0.6140
Epoch 7/100
7000/7000 [==============================] - 76s 11ms/step - loss: 0.4728 - val_loss: 0.5887
Epoch 8/100
7000/7000 [==============================] - 75s 11ms/step - loss: 0.4525 - val_loss: 0.5781
Epoch 9/100
7000/7000 [==============================] - 76s 11ms/step - loss: 0.4347 - val_loss: 0.5599
Epoch 10/100
7000/7000 [==============================] - 76s 11ms/step - loss: 0.4180 - val_loss: 0.5511
Epoch 11/100
7000/7000 [==============================

In [58]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 100)    0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 116)    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 365568      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 256),  381952      decoder_inputs[0][0]             
                                                                 encoder_lstm[0][1]               
          

In [44]:
#Model to Test

In [45]:
# Encoder inference model
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder inference model

decoder_state_input_h = Input(shape=(HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(HIDDEN_UNITS,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_inputs] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [46]:
max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

In [47]:
def predict_sent(input_text):
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            emb = unknown_emb
            if word in word2em:
                emb = word2em[word]
            input_wids.append(emb)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, max_encoder_seq_length)
        states_value = encoder_model_inf.predict(input_seq)
        target_seq = np.zeros((1, 1,num_decoder_tokens))
        target_seq[0, 0, target_word2idx['\t']] = 1
        target_text = ''
        terminated = False
        while not terminated:
            output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = target_idx2word[sample_token_idx]
            target_text += sample_word

            if sample_word == '\n' or len(target_text) >= max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

In [48]:
print(predict_sent('He is sick.'))

Il est malade.
